# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 2:47PM,238569,21,616304,"NBTY Global, Inc.",Released
1,Jun 1 2022 2:44PM,238568,20,R1-8091832,Reliable 1 Laboratories LLC,Released
2,Jun 1 2022 2:29PM,238567,19,8091851,"Emersa Waterbox, LLC",Released
3,Jun 1 2022 1:55PM,238563,10,MET210809,"Methapharm, Inc.",Released
4,Jun 1 2022 1:54PM,238562,10,MET210801,"Methapharm, Inc.",Released
5,Jun 1 2022 1:54PM,238562,10,MET210802,"Methapharm, Inc.",Released
6,Jun 1 2022 1:54PM,238562,10,MET210804,"Methapharm, Inc.",Released
7,Jun 1 2022 1:54PM,238562,10,MET210788,"Methapharm, Inc.",Released
8,Jun 1 2022 1:54PM,238562,10,MET210778,"Methapharm, Inc.",Released
9,Jun 1 2022 1:54PM,238562,10,MET210776,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,238562,Released,7
46,238563,Released,1
47,238567,Released,1
48,238568,Released,1
49,238569,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238562,NaN,NaN,7.0
238563,NaN,NaN,1.0
238567,NaN,NaN,1.0
238568,NaN,NaN,1.0
238569,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0.0,13.0,7.0
238433,0.0,15.0,8.0
238436,0.0,0.0,2.0
238438,0.0,0.0,1.0
238443,18.0,16.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0,13,7
238433,0,15,8
238436,0,0,2
238438,0,0,1
238443,18,16,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,0,13,7
1,238433,0,15,8
2,238436,0,0,2
3,238438,0,0,1
4,238443,18,16,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,,13,7
1,238433,,15,8
2,238436,,,2
3,238438,,,1
4,238443,18,16,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 2:47PM,238569,21,"NBTY Global, Inc."
1,Jun 1 2022 2:44PM,238568,20,Reliable 1 Laboratories LLC
2,Jun 1 2022 2:29PM,238567,19,"Emersa Waterbox, LLC"
3,Jun 1 2022 1:55PM,238563,10,"Methapharm, Inc."
4,Jun 1 2022 1:54PM,238562,10,"Methapharm, Inc."
11,Jun 1 2022 1:52PM,238560,10,Emerginnova
13,Jun 1 2022 1:50PM,238559,10,"Methapharm, Inc."
29,Jun 1 2022 1:42PM,238557,12,Hush Hush
34,Jun 1 2022 1:40PM,238558,10,"Nextsource Biotechnology, LLC"
35,Jun 1 2022 1:39PM,238556,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 1 2022 2:47PM,238569,21,"NBTY Global, Inc.",,,1
1,Jun 1 2022 2:44PM,238568,20,Reliable 1 Laboratories LLC,,,1
2,Jun 1 2022 2:29PM,238567,19,"Emersa Waterbox, LLC",,,1
3,Jun 1 2022 1:55PM,238563,10,"Methapharm, Inc.",,,1
4,Jun 1 2022 1:54PM,238562,10,"Methapharm, Inc.",,,7
5,Jun 1 2022 1:52PM,238560,10,Emerginnova,,,2
6,Jun 1 2022 1:50PM,238559,10,"Methapharm, Inc.",,,16
7,Jun 1 2022 1:42PM,238557,12,Hush Hush,,,5
8,Jun 1 2022 1:40PM,238558,10,"Nextsource Biotechnology, LLC",,,1
9,Jun 1 2022 1:39PM,238556,12,Hush Hush,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 2:47PM,238569,21,"NBTY Global, Inc.",1,,
1,Jun 1 2022 2:44PM,238568,20,Reliable 1 Laboratories LLC,1,,
2,Jun 1 2022 2:29PM,238567,19,"Emersa Waterbox, LLC",1,,
3,Jun 1 2022 1:55PM,238563,10,"Methapharm, Inc.",1,,
4,Jun 1 2022 1:54PM,238562,10,"Methapharm, Inc.",7,,
5,Jun 1 2022 1:52PM,238560,10,Emerginnova,2,,
6,Jun 1 2022 1:50PM,238559,10,"Methapharm, Inc.",16,,
7,Jun 1 2022 1:42PM,238557,12,Hush Hush,5,,
8,Jun 1 2022 1:40PM,238558,10,"Nextsource Biotechnology, LLC",1,,
9,Jun 1 2022 1:39PM,238556,12,Hush Hush,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 2:47PM,238569,21,"NBTY Global, Inc.",1,,
1,Jun 1 2022 2:44PM,238568,20,Reliable 1 Laboratories LLC,1,,
2,Jun 1 2022 2:29PM,238567,19,"Emersa Waterbox, LLC",1,,
3,Jun 1 2022 1:55PM,238563,10,"Methapharm, Inc.",1,,
4,Jun 1 2022 1:54PM,238562,10,"Methapharm, Inc.",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 2:47PM,238569,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 1 2022 2:44PM,238568,20,Reliable 1 Laboratories LLC,1.0,NaN,NaN
2,Jun 1 2022 2:29PM,238567,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,Jun 1 2022 1:55PM,238563,10,"Methapharm, Inc.",1.0,NaN,NaN
4,Jun 1 2022 1:54PM,238562,10,"Methapharm, Inc.",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 2:47PM,238569,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 1 2022 2:44PM,238568,20,Reliable 1 Laboratories LLC,1.0,0.0,0.0
2,Jun 1 2022 2:29PM,238567,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,Jun 1 2022 1:55PM,238563,10,"Methapharm, Inc.",1.0,0.0,0.0
4,Jun 1 2022 1:54PM,238562,10,"Methapharm, Inc.",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2862473,79.0,34.0,14.0
102,715568,2.0,1.0,0.0
12,954150,67.0,2.0,0.0
15,1430558,40.0,44.0,18.0
16,1192681,5.0,1.0,0.0
18,238522,1.0,0.0,0.0
19,954192,4.0,0.0,0.0
20,1192697,39.0,12.0,0.0
21,238569,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2862473,79.0,34.0,14.0
1,102,715568,2.0,1.0,0.0
2,12,954150,67.0,2.0,0.0
3,15,1430558,40.0,44.0,18.0
4,16,1192681,5.0,1.0,0.0
5,18,238522,1.0,0.0,0.0
6,19,954192,4.0,0.0,0.0
7,20,1192697,39.0,12.0,0.0
8,21,238569,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,79.0,34.0,14.0
1,102,2.0,1.0,0.0
2,12,67.0,2.0,0.0
3,15,40.0,44.0,18.0
4,16,5.0,1.0,0.0
5,18,1.0,0.0,0.0
6,19,4.0,0.0,0.0
7,20,39.0,12.0,0.0
8,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,102,Released,2.0
2,12,Released,67.0
3,15,Released,40.0
4,16,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,14.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
Executing,34.0,1.0,2.0,44.0,1.0,0.0,0.0,12.0,0.0
Released,79.0,2.0,67.0,40.0,5.0,1.0,4.0,39.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,14.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,1.0,2.0,44.0,1.0,0.0,0.0,12.0,0.0
2,Released,79.0,2.0,67.0,40.0,5.0,1.0,4.0,39.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,14.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
1,Executing,34.0,1.0,2.0,44.0,1.0,0.0,0.0,12.0,0.0
2,Released,79.0,2.0,67.0,40.0,5.0,1.0,4.0,39.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()